In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import agent
import h5py
import matplotlib as mpl, matplotlib.pyplot as plt
from matplotlib import animation, rc

import os
import sys
if os.path.isdir('/opt/ros/kinetic/lib/python2.7/dist-packages'):
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

from IPython.display import HTML

In [ ]:
dist = lambda x,y: np.sqrt(np.sum((x[:2] - y[:2])**2))

a1_gaze = lambda start, end: np.vstack([ # exploration
    gaze.fixate(),
    gaze.saccade_to(start),
    gaze.fixate(),
    gaze.saccade_to(end),
])
a2_gaze = lambda x: gaze.pursue(x) # pursuit
a3_gaze = lambda x: gaze.pursue(x) # toggle
a4_gaze = lambda start, end, num_frames: np.vstack([ # correction
    gaze.saccade_to(start, num_frames//3+(num_frames-(num_frames//3)*3)),
    gaze.fixate(num_frames//3),
    gaze.saccade_to(end, num_frames//3),
])
a5_gaze = lambda x: gaze.pursue(x) # mode switch

a1_joy = lambda x: joy.hold(x) # exploration
a2_joy = lambda x: joy.move_to(x) # pursuit
a3_joy = lambda x,te,tl: joy.toggle_to(x,te,tl) # toggle
a4_joy = lambda x: joy.move_to(x) # correction
a5_joy = lambda start, end, num_frames: np.vstack([ # mode switch
    joy.move_to(start, num_frames),
    joy.move_to(end, num_frames),
])
num_vids = 0
while num_vids < 1000:
# for j in range(1000):
    start_pos = np.random.uniform(1,9,2)
    goal = np.random.uniform(1,9,2)
    
    while dist(start_pos, goal) < 5:
        start_pos = np.random.uniform(1,9,2)
        goal = np.random.uniform(1,9,2)

    gaze_params = {
        'mu_vel': np.random.uniform(3,10)*.1, 
        'sigma_vel': np.random.uniform(0,1)*.1, 
        'mu_fix': np.random.uniform(30,90), 
        'sigma_fix': np.random.uniform(0,.5)*.1, 
        'sigma_x': np.random.uniform(0,.3), 
        'sigma_y': np.random.uniform(0,.3), 
        'position': start_pos,
        'heading': None,
    }

    joy_params = {
        'mu_vel': np.random.uniform(1,3)*.1, 
        'sigma_vel': np.random.uniform(0,1)*.1, 
        'position': start_pos,
        'heading': None,
    }

    gaze = agent.GazeAgent(**gaze_params)
    joy = agent.JoyAgent(**joy_params)
    
    move_order = np.random.choice(np.arange(*goal.shape), replace=False, size=2)
    counter = 0
    i=0
    action_ls = []
    while dist(joy.position, goal) > .5:
        if counter % 2 == 0:
            init_pos = np.array(gaze.position)
            action1_gaze = a1_gaze(goal, init_pos)
            action1_joy = a1_joy(action1_gaze.shape[0])
            action_ls.append(np.hstack((action1_gaze, action1_joy, np.zeros((action1_joy.shape[0],1)))))
            
            ## DO mode switch here
            move_to_d1 = (goal[:2]*np.abs((1-(1-(i%2))-move_order)) - joy.position[:2]*np.abs((1-(1-(i%2))-move_order)))
            move_to_d1 += joy.position[:2]*np.abs((1-(1-(i%2))-move_order))
            
            move_to_d2 = joy.position[:2]*np.abs((1-((i%2))-move_order))
            move_to = move_to_d1 + move_to_d2 - action1_joy[-1, :2]
            
            sign = np.array([t/np.abs(t) if t != 0 else 0 for t in move_to])
            mags = np.array([min(np.abs(t), 2) for t in move_to])
            move_to = sign*mags + action1_joy[-1, :2]
            action5_joy = a5_joy(move_to, action1_joy[-1], 10)
            action5_gaze = a5_gaze(action5_joy)
            action_ls.append(np.hstack((action5_gaze, action5_joy, np.ones((action5_joy.shape[0],1))*3)))
        else:
            how_far = np.random.randint(1,3)
            move_to_d1 = (goal[:2]*np.abs((1-(1-(i%2))-move_order)) - joy.position[:2]*np.abs((1-(1-(i%2))-move_order)))/how_far
            move_to_d1 += joy.position[:2]*np.abs((1-(1-(i%2))-move_order))
            
            move_to_d2 = joy.position[:2]*np.abs(((1-(i%2))-move_order))
            move_to = move_to_d1 + move_to_d2
            
            val = np.random.rand()
            if val <= 1/3:
                action2_joy = a2_joy(move_to)
                action2_gaze = a2_gaze(action2_joy)
                action_ls.append(np.hstack((action2_gaze, action2_joy, np.ones((action2_joy.shape[0],1))*2)))
            elif val > 1/3 and val <= 2/3:
                te = np.random.randint(4,10)
                tl = np.random.randint(4,10)
                action3_joy = a3_joy(move_to, te, tl)
                action3_gaze = a3_gaze(action3_joy)
                action_ls.append(np.hstack((action3_gaze, action3_joy, np.ones((action3_joy.shape[0],1))*4)))
            else:
                action4_joy = a4_joy(move_to)
                if action4_joy.shape[0] > 3:
                    action4_gaze = a4_gaze(goal, action4_joy[-1], action4_joy.shape[0])
                else:
                    action4_gaze = a2_gaze(action4_joy)
                action_ls.append(np.hstack((action4_gaze, action4_joy, np.ones((action4_joy.shape[0],1))*1)))
                
            i+=1
        counter+=1
        
    action = np.vstack(action_ls)
        
    gaze_ims = np.zeros((action.shape[0], 64*64))
    joy_ims = np.zeros((action.shape[0], 64*64))
    for i,a in enumerate(action):
        im_gaze = np.zeros((64,64),dtype="uint8")
        im_joy = np.zeros((64,64),dtype="uint8")

        cv2.circle(im_gaze, (int(goal[0]/10*64), 64-int(goal[1]/10*64)), 4, (255), -1, cv2.LINE_AA)
        rect_size = 4
        pt1 = np.array((np.round(a[0]/10*64)-rect_size, 64-np.round(a[1]/10*64)+rect_size))#+ np.array((off,-off))
        pt2 = np.array((np.round(a[0]/10*64)+rect_size, 64-np.round(a[1]/10*64)-rect_size))# + np.array((off,-off))
        cv2.rectangle(im_gaze, tuple(pt1.astype(int)), tuple(pt2.astype(int)), (255), -1, 8)

        cv2.circle(im_joy, (int(goal[0]/10*64), 64-int(goal[1]/10*64)), 4, (255), -1, cv2.LINE_AA)
        pt1 = (a[3]/10*64, 64-a[4]/10*64-5)
        pt2 = (a[3]/10*64-4, 64-a[4]/10*64+3)
        pt3 = (a[3]/10*64+4, 64-a[4]/10*64+3)    
        triangle_cnt = np.array( [pt1, pt2, pt3] ).reshape((-1,1,2)).astype(int)
        cv2.drawContours(im_joy, [triangle_cnt], 0, (255), -1, cv2.LINE_AA)
        gaze_ims[i] = im_gaze.reshape(-1) / 255
        joy_ims[i] = im_joy.reshape(-1)/255
    
    if action.shape[0] < 1000 and action.shape[0] > 100:
        print('actreact_h5_data2/{:05d}.h5'.format(num_vids))
        hf = h5py.File('actreact_h5_data2/{:05d}.h5'.format(num_vids), 'w')
        hf.create_dataset('poses', data=action)
        hf.create_dataset('gaze_ims', data=gaze_ims)
        hf.create_dataset('joy_ims', data=joy_ims)
        hf.create_dataset('goals', data=goal)
        hf.close()
        np.savez_compressed(
            'actreact_h5_data2/{:05d}_params.npz'.format(num_vids), 
            gaze_params=gaze_params,
            joy_params=joy_params,
        )

        num_vids+=1
    

In [ ]:
plt.rc('animation', html='html5')

vid_num = int(np.random.choice(1000))
print(vid_num)
with h5py.File('actreact_h5_data2/{:05d}.h5'.format(vid_num), 'r') as data:
    gvid = data['gaze_ims'][:].reshape(-1, 64, 64)
    jvid = data['joy_ims'][:].reshape(-1, 64, 64)

    
fig, axs = plt.subplots(1, 1)
axs.set_xticks([])
axs.set_yticks([])
x = gvid.shape[1]
y = gvid.shape[2]

im1 = axs.imshow(gvid[0], cmap='gray')
def init2():
    im1.set_data(gvid[0]+jvid[0])
    return (im1,)

# animation function. This is called sequentially
def animate2(i):
    im1.set_data(gvid[i]+jvid[i])
    return (im1,)

# call the animator. blit=True means only re-draw the parts that 
# have changed.
movie = animation.FuncAnimation(
    fig, 
    animate2, 
    init_func=init2,
    frames=gvid.shape[0], 
    interval=(1/120)*1000, 
    blit=True
)

movie

In [ ]:
from matplotlib import rc
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}
rc('font',**font)
rc('text', usetex=True)

fig, axs = plt.subplots(1,6, figsize=(20,3))
with h5py.File('actreact_h5_data2/{:05d}.h5'.format(100), 'r') as data:
    split = 19
    for j in range(6):
        img = data['gaze_ims'][split*j].reshape(64,64)+data['joy_ims'][split*j].reshape(64,64)
        img = img.clip(0,1)
        axs[j].imshow(img, cmap='gray')
        axs[j].spines['right'].set_visible(False)
        axs[j].spines['top'].set_visible(False)
        axs[j].spines['bottom'].set_visible(False)
        axs[j].spines['left'].set_visible(False)
        axs[j].set_xticks([])
        axs[j].set_yticks([])
        axs[j].set_xlabel(r'Frame {}'.format(j*split))
    plt.show()
        
fig.savefig('synthexample.pdf', bbox_inches='tight')